In [6]:
import geopandas as gpd
from shapely.geometry import Point
from pyproj import CRS
import os
import zipfile

# Sodankyla coordinates (latitude, longitude)
sodankyla_coords = (67.3616, 26.6342)

# Create a GeoDataFrame with a single point representing Sodankyla
sodankyla_point = gpd.GeoDataFrame(geometry=[Point(sodankyla_coords)], crs="EPSG:4326")  # WGS 84

# Reproject to a projected CRS (UTM zone for example)
projected_crs = "EPSG:32635"  # UTM zone 35N, you may need to choose the appropriate UTM zone for your area
sodankyla_point = sodankyla_point.to_crs(projected_crs)

# Create a buffer around Sodankyla
buffer_radius_km = 5
buffer_gdf = sodankyla_point.copy()
buffer_gdf['geometry'] = buffer_gdf.buffer(buffer_radius_km * 1000)  # Buffer in meters

# Reproject back to geographic CRS for saving to shapefile
buffer_gdf = buffer_gdf.to_crs("EPSG:4326")

# Specify the folder name
output_folder = 'sodankyla_shapefile'

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Save the GeoDataFrame to a shapefile in the specified folder
shapefile_path = os.path.join(output_folder, "sodankyla_buffer.shp")
buffer_gdf.to_file(shapefile_path)

# Zip the shapefile files
zipfile_path = os.path.join(output_folder, "sodankyla_buffer.zip")
with zipfile.ZipFile(zipfile_path, 'w') as zipf:
    for root, _, files in os.walk(shapefile_path):
        for file in files:
            file_path = os.path.join(root, file)
            arcname = os.path.relpath(file_path, shapefile_path)
            zipf.write(file_path, arcname=arcname)

print(f"Shapefile and associated files zipped at: {zipfile_path}")


Shapefile and associated files zipped at: sodankyla_shapefile/sodankyla_buffer.zip


In [23]:
import geopandas as gpd
from shapely.geometry import Point
import simplekml
import os

# Create a folder named 'shapefiles'
output_folder = 'shapefiles'
os.makedirs(output_folder, exist_ok=True)

# Sodankylä coordinates
sodankyla_coords = (26.5900, 67.3662)  # Correct order: (longitude, latitude)

# Create a GeoDataFrame with a point for Sodankylä and set the CRS explicitly
sodankyla_point = gpd.GeoDataFrame(geometry=[Point(sodankyla_coords)], crs="EPSG:4326")

# Reproject to a projected CRS (e.g., UTM)
sodankyla_point_utm = sodankyla_point.to_crs(epsg=32633)  # You may need to choose an appropriate EPSG code

# Create a buffer of 5km around Sodankylä
buffer_radius = 5000  # in meters
sodankyla_buffer_utm = sodankyla_point_utm.buffer(buffer_radius)

# Reproject back to the original CRS
sodankyla_buffer = sodankyla_buffer_utm.to_crs(epsg=4326)

# Print CRS information for verification
print("Original CRS:", sodankyla_point.crs)
print("Buffered CRS:", sodankyla_buffer.crs)

# Save the buffered geometry as a shapefile
shapefile_filename = os.path.join(output_folder, "sodankyla_5.shp")
sodankyla_buffer.to_file(shapefile_filename)

# Create a KML object
kml = simplekml.Kml()

# Create a polygon from the buffered geometry
polygon_coords = [(x, y) for x, y in sodankyla_buffer.iloc[0].exterior.coords]
polygon = kml.newpolygon(name="5km Radius around Sodankylä", outerboundaryis=polygon_coords)

# Save the KML file
kml_filename = os.path.join(output_folder, "sodankyla_5.kml")
kml.save(kml_filename)

print(f"Shapefile '{shapefile_filename}' and KML file '{kml_filename}' created.")


Original CRS: EPSG:4326
Buffered CRS: EPSG:4326
Shapefile 'shapefiles/sodankyla_5.shp' and KML file 'shapefiles/sodankyla_5.kml' created.
